# TDOST-Based HAR Model using Aruba Dataset
This notebook implements the layout-agnostic Human Activity Recognition model using the TDOST methodology as described in the provided document.

In [2]:
%pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 MB 9.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 28.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 36.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 50.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 kB 21.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.utils import to_categorical

## Step 1: Load and Preprocess Data

In [11]:
"""
def load_aruba_data(filepath):
    columns = ['Date', 'Time', 'Sensor', 'Value', 'Activity', 'Begin_End']
    data = pd.read_csv(filepath, sep=" ", header=None, names=columns, engine='python',
                       parse_dates=[[0, 1]], keep_date_col=False)
    data['Activity'] = data['Activity'].fillna(method='ffill')
    data['Begin_End'] = data['Begin_End'].fillna('')
    return data

aruba_data = load_aruba_data('/Users/harrisonkirstein/Desktop/CSCI-4380-Honors-Option-Repo/CSCI 4380 Honors Option Project/Datasets/aruba/data')
aruba_data.head()

"""


# Step 1: Load and Preprocess Dataset

# Define file path
aruba_data_path = '/Users/harrisonkirstein/Desktop/CSCI-4380-Honors-Option-Repo/CSCI 4380 Honors Option Project/Datasets/aruba/data'

# Load dataset with variable columns
aruba_data = pd.read_csv(
    aruba_data_path, 
    header=None, 
    names=['Date', 'Time', 'Sensor', 'Value', 'Activity', 'Begin_End'], 
    delim_whitespace=True,
    engine='python'
)

# Combine Date and Time into a single timestamp column
aruba_data['Date_Time'] = pd.to_datetime(aruba_data['Date'] + ' ' + aruba_data['Time'], errors='coerce')
aruba_data.drop(columns=['Date', 'Time'], inplace=True)

# Fill missing columns with NaN for rows without activity labels
aruba_data.fillna('', inplace=True)

# Preview the dataset
aruba_data.head()

/var/folders/4s/ccclfjqs2v504c8lk55hwgj40000gn/T/ipykernel_58512/2578245890.py:22: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  aruba_data = pd.read_csv(


,Sensor,Value,Activity,Begin_End,Date_Time
0,M003,ON,Sleeping,begin,2010-11-04 00:03:50.209589
1,M003,OFF,,,2010-11-04 00:03:57.399391
2,T002,21.5,,,2010-11-04 00:15:08.984841
3,T003,21,,,2010-11-04 00:30:19.185547
4,T004,21,,,2010-11-04 00:30:19.385336


## Step 2: Generate TDOST Descriptions

In [16]:
# Generate TDOST descriptions
def generate_tdost(data):
    descriptions = []
    for i, row in data.iterrows():
        if row['Activity']:
            desc = f"{row['Sensor']} sensor in {row['Value']} triggered at {row['Date_Time'].strftime('%H:%M:%S')}."
            descriptions.append(desc)
    return descriptions

aruba_data['TDOST_Basic'] = generate_tdost(aruba_data)
aruba_data.head()

ValueError: Length of values (12954) does not match length of index (1719558)

## Step 3: Encode TDOST Descriptions and Labels

In [ ]:
# Load pre-trained Sentence Transformer
model = SentenceTransformer('all-distilroberta-v1')

# Encode TDOST descriptions
aruba_data['Encoded_TDOST'] = aruba_data['TDOST'].apply(lambda x: model.encode(x) if isinstance(x, str) else None)

# Encode labels
label_encoder = LabelEncoder()
aruba_data['Encoded_Activity'] = label_encoder.fit_transform(aruba_data['Activity'])
num_classes = len(label_encoder.classes_)
aruba_data.head()

## Step 4: Train-Test Split

In [ ]:
# Split data
X = np.stack(aruba_data['Encoded_TDOST'].dropna())
y = to_categorical(aruba_data['Encoded_Activity'].dropna(), num_classes=num_classes)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Step 5: Build and Train the Model

In [ ]:
# Build LSTM model
def build_model(input_shape, num_classes):
    model = Sequential([
        Bidirectional(LSTM(64, return_sequences=False), input_shape=input_shape),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

input_shape = (X_train.shape[1],)
model = build_model(input_shape, num_classes)

# Train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

## Step 6: Evaluate and Test the Model

In [ ]:
# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Test on new data (replace with actual test data)
new_sample = "motion sensor in bedroom triggered at 08:15:23."
new_embedding = model.encode(new_sample)
prediction = model.predict(np.array([new_embedding]))
predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])
print(f"Predicted Activity: {predicted_label[0]}")